In [ ]:
#python3 -m sglang.launch_server --model-path Qwen/Qwen2.5-1.5B-Instruct --port 16161 --disable-cuda-graph --quantization fp8 --disable-radix-cache

In [1]:
from sglang_infer import chat, complete

In [16]:
PROMPT_QUESTION_GENERATION = """<|im_start|>system
You are a helpful assistant. The user ends its message with "<|im_end|>" suffix.<|im_end|>
<|im_start|>user
"""

def generate_prompt():
    return complete(PROMPT_QUESTION_GENERATION, temperature=1)

In [17]:
from tqdm.auto import tqdm

ds = []

for i in tqdm(range(1000)):
    prompt = generate_prompt()
    responses = []
    for j in range(2): #TODO: change to 3
        response = chat(prompt, temperature=1)
        responses.append(response)
    ds.append({"prompt": prompt, "responses": responses})

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
import json
len(ds)
with open("ds.json", "w") as f:
    f.write(json.dumps(ds))

In [19]:
import json
ds = json.loads(open("ds.json").read())

In [20]:
from reward import reward
rewards = reward(ds[0]["prompt"], ds[0]["responses"])
print(rewards)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[-9.5625, -9.375]


In [21]:
from tqdm.auto import tqdm

ds_with_rewards = []

for example in tqdm(ds):
    new_example = {
        "prompt": example["prompt"],
        "responses": example["responses"],
        "reward": reward(example["prompt"], example["responses"]),
    }
    ds_with_rewards.append(new_example)

ds = ds_with_rewards

  0%|          | 0/600 [00:00<?, ?it/s]

In [22]:
print(ds[0]["prompt"])
print("##########################################################################")
print(ds[0]["responses"][0])
print("--- reward: ", ds[0]["reward"][0])
print("--------------------------------------------------------------------------")
print(ds[0]["responses"][1])
print("--- reward: ", ds[0]["reward"][1])
print("--------------------------------------------------------------------------")
#print(ds[0]["responses"][2])
#print("--- reward: ", ds[0]["reward"][2])
#print("--------------------------------------------------------------------------")

I'm having trouble getting excited about my job. How can I feel more passionate about what I do?

I don't have enough information to directly address your concerns as I'm not a direct provider of personal advice or therapeutic support. However, I can offer some general suggestions that might be helpful:

1. **Identify What You Alike About Your Job**: Spend some time understanding what aspects of your job you enjoy. It could be the people you work with, the tasks you enjoy, the challenges you face, or the final product.

2. **Reflect on Your Goals**: Consider what your long-term goals are related to your job. If they align more closely with what you enjoy, you might feel more excited about the work.

3. **Set Achievable Goals**: Set small, achievable goals. Completing these goals on time can help you feel engaged and passionate as you finish them.

4. **Stay Motivated**: Keep working on the tasks that excite you, and make time for the ones that don’t. Learning new skills can also make y

In [25]:
preference_ds = []

for example in ds:
    best = example["responses"][example["reward"].index(max(example["reward"]))]

    preference_ds.append({
            "prompt": example["prompt"],
            "response": best,
    })

In [26]:
from datasets import Dataset

ds = Dataset.from_list(preference_ds)
ds

Dataset({
    features: ['prompt', 'response'],
    num_rows: 600
})

In [27]:
from huggingface_hub import notebook_login
notebook_login()

In [28]:
ds.push_to_hub("OpenEndedLM/OpenEnded-v0.1")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/OpenEndedLM/OpenEnded-v0.1/commit/29d2c9fe5f96337a5c8879eaf2a85b2424e5d91d', commit_message='Upload dataset', commit_description='', oid='29d2c9fe5f96337a5c8879eaf2a85b2424e5d91d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/OpenEndedLM/OpenEnded-v0.1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='OpenEndedLM/OpenEnded-v0.1'), pr_revision=None, pr_num=None)